In [2]:
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
dataset='/content/drive/MyDrive/Datasets'

In [4]:
IMAGE_SIZE=224
BATCH_SIZE=16

#pre=processing
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
    )

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.2
)

train_datagen=train_datagen.flow_from_directory(
    dataset,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

test_datagen=test_datagen.flow_from_directory(
    dataset,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 3198 images belonging to 6 classes.
Found 796 images belonging to 6 classes.


In [5]:
  # useful for getting number of output classes
folders = glob('/content/drive/MyDrive/Datasets/*')


In [6]:
IMAGE_SIZE = [224, 224]

In [7]:
# Here we will be using imagenet weights

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [8]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [9]:
from tensorflow.keras.layers import MaxPooling2D
# our layers - you can add more if you want
x = MaxPooling2D(pool_size=(2, 2),strides=2)(vgg.output)

In [10]:
x = Flatten()(x)

In [11]:
from tensorflow.keras.layers import Dropout
x = Dropout(0.5)(x)

In [12]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)


In [13]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
from tensorflow.keras.optimizers import Adam

In [15]:
# tell the model what cost and optimization method to use
learning_rate=0.01
Epochs=20

opt=Adam(learning_rate=learning_rate,decay=learning_rate/Epochs)

model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  train_datagen,
  validation_data=test_datagen,
  epochs=Epochs,
  steps_per_epoch=len(train_datagen),
  validation_steps=len(test_datagen)
)

Epoch 1/20
  1/200 [..............................] - ETA: 1:13:34 - loss: 2.8191 - accuracy: 0.0000e+00

In [ ]:
import matplotlib.pyplot as plt